In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import PIL
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [3]:
df['labels'] = df['labels'].apply(lambda s: s.split(' '))

In [4]:
datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.15)
pos = ImageDataGenerator(rescale = 1./255)
train_generator = datagen.flow_from_dataframe(
dataframe = df,
directory="../input/resized-plant2021/img_sz_512/",
x_col="image",
y_col="labels",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
color_mode = "rgb",
target_size=(224,224))

val_generator =  datagen.flow_from_dataframe(
dataframe = df,
directory="../input/resized-plant2021/img_sz_512/",
x_col="image",
y_col="labels",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
color_mode = "rgb",
target_size=(224,224))

Found 15838 validated image filenames belonging to 6 classes.
Found 2794 validated image filenames belonging to 6 classes.


In [5]:
test_data = pos.flow_from_dataframe(
    submission,
    directory = '../input/plant-pathology-2021-fgvc8/test_images',
    x_col="image",
    y_col= "labels",
    color_mode="rgb",
    target_size = (224,224),
    classes=None,
    class_mode=None,
    batch_size=8,
    shuffle=False,
    seed=40,
)

Found 3 validated image filenames.


In [6]:
import tensorflow_addons as tfa
number_of_classes = 6
threshold_x=0.25
f1 = tfa.metrics.F1Score(num_classes=6, average='macro',threshold = threshold_x)
dependencies = {
    'f1': f1
}

In [7]:
model = tf.keras.models.load_model('../input/ppresearch02densenet121/model-best.h5')

In [8]:
preds = model.predict(test_data)

In [9]:
preds = preds.tolist()

indices = []
for pred in preds:
    temp = []
    for category in pred:
        if category>=0.25:
            temp.append(pred.index(category))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)
    
print(indices)

[[0], [0, 1, 5], [1]]


In [10]:
labels = (train_generator.class_indices)
labels = { v:k for k,v in labels.items() }

testlabels = []


for image in indices:
    temp = []
    for i in image:
        temp.append(str(labels[i]))
    testlabels.append(' '.join(temp))

print(testlabels)

['complex', 'complex frog_eye_leaf_spot scab', 'frog_eye_leaf_spot']


In [11]:
submission['labels'] = testlabels

In [12]:
submission.to_csv('submission.csv', index = False)